# 1. Main Workflow

This notebook describes the main workflow of normative modelling (NM). It contains the accompanying code for the manuscript: ___, but can also be used as a standalone learning tool for NM.

## 1.0 Setting up the environment

-----

<details>
<summary><b>System requirements</b></summary>

To follow this guide, you will need:
- Either a Mac, Linux, or Windows machine with WSL (Windows Subsystem for Linux) installed.
- About ___ GB of free disk space (for the data and the environment)
- About ___ GB of RAM (more is better, but not required)
- A decent processor (we recommend a multi-core CPU)
- A working internet connection (for installing the packages and downloading the data)

For Windows users with WSL, everything that follows will be done in the WSL terminal. You will have to follow the installation instructions for Linux/Mac. 

</details>

-----

<details>
<summary><b>Setting up Anaconda</b></summary>

Anaconda is a free and open-source distribution of the Python and R programming languages for scientific computing, that aims to simplify package management and deployment. We highly recommend using Anaconda (or an alternative like miniconda or venv) to manage your python environment. We will use Anaconda in this tutorial.

Follow the instructions on the anaconda website: https://www.anaconda.com/docs/getting-started/anaconda/install

Verify the installation by running the following command in your terminal:

```bash
conda --version
```

If you see the version number, you are good to go.

</details>

-----

<details>
<summary><b>Creating an environment and install the PCNtoolkit</b></summary>

Choose an environment name, here we will use `ptk` as a placeholder.

```bash
conda create -n ptk python=3.12
conda activate ptk
pip install pcntoolkit==1.1.0
conda install -c conda-forge jupyter ipykernel ipywidgets
```

Now to verify that the installation is successful, run the following code:

```bash
python -c "import pcntoolkit as pcn; print(pcn.__version__)" 
```

This should print `1.1.0` without any errors.

</details>

-----

## 1.1 Data Selection

-----

<details>
<summary><b>Data inclusion criteria</b></summary>

A large reference - or normative - set is required to train a well-calibrated normative model. There are two types of uncertainty that have to be considered when building a normative model; epistemic and aleatoric uncertainty. The epistemic uncertainty is the uncertainty in our knowledge, while the aleatoric uncertainty is due to the natural variation in the population. The latter is what we aim to capture with our normative models, and to model it as accurately as possible, the former has to be minimized. As the epistemic uncertainty decreases with the size of the training data, it is desired to have a large reference set.

What can be considered a reference set depends on the research question. If the goal is to parse deviations from a healthy population, the reference set should include only healthy participants. If the goal is to parse heterogeneity within a clinical cohort, one could choose to use a set of participants with that particular condition as the reference set. For instance, if one is interested to parse heterogeneity within a clinical cohort of patients with major depressive disorder (MDD), the reference set could include MDD patients who have responded well to medication. 

In our example, we will use image-derive phenotypes FCON1000 dataset, which contains data from 1078 healthy subjects from _ different sites. We have included the full dataset in the `data/fcon` folder, but it can also be downloaded from the [FCON1000 website](https://fcon_1000.projects.nitrc.org/indi/fcp_abcd/abcd.html).

</details>

-----

In [ ]:
import pandas as pd
try:
    df = pd.read_csv('../data/fcon/fcon1000.csv')
except FileNotFoundError:
    # Load the data from the 
    df = pd.read_csv('https://raw.githubusercontent.com/predictive-clinical-neuroscience/PCNtoolkit-demo/refs/heads/main/data/fcon1000.csv')


Loading data from local file



<details>
<summary><b>Covariates, batch effects, and response variables</b></summary>

As we will see, the input data generally consist of a number of covariates, a number of batch effect dimensions, and a number of response variables. The covariates are typically continuous variables, like age and height, but can also be numerically encoded categorical variables, like sex. The batch effect dimensions are categorical variables, scan site or scanner brand/model. The response variables are continuous variables, like brain volumes or brain activity patterns. All these variables must be available for all subjects in the reference set. 

Normative models made using the PCNtoolkit version 1.1.0 all consist of one or more linear regressions on the covariates, and a mechanism to handle batch effects, which depends on the type of model. Which variables to select as covariates and batch effects depends on the research question, but it is generally best to include variables that are known to be relevant for the research question. It is important to understand that the goal in normative modelling is to get deviation scores that are as independent as possible from the covariates and batch effects. In an example scenario, if we want to study the effect of meditation on , like age, sex, and brain activity patterns, but not include variables that are  like education level or income.

Sufficient care must be taken to ensure that all data sources adhere to the same schema. For instance, if the reference set includes certain clinical variables, like age and height,these should be available and expressed in consistent units in all other data sources. Furthermore, the PCNtoolkit demands that any data for which predictions are to be made, are from batches that were present in the training set. I.e., a model which captures site effects can not make predictions on data from a site which was not present in the training set.

</details>


-----

<details>
<summary><b>Train-test split</b></summary>

The reference set is split into a train set and a test set. The train set is used to train the normative model, while the test set is used to assess the performance of the model. A trade-off has to be made between the size of the train set and the size of the test set. A larger train set will result in a more accurate normative model, but a larger test set will result in a more reliable performance assessment. We recommend using a 80%-20% or 70%-30% split for the train and test set, respectively.

For a correct performance assessment, the test set should be as statistically similar as possible to the train set. Stratification of the train-test split ensures that the distribution of the batch effect values in the train and test set are similar. 


</details>

-----